In [1]:
import os 
import importlib

path='/home/enis/projects/nna/src/scripts/'
os.chdir(path)
import teacher

import itertools
import pandas as pd

Output(layout=Layout(border='1px solid black'))

In [2]:
importlib.reload(teacher)
config = teacher.setup()

## data-1: as many sampes as possible
config['new_dataset_path'] = './datasetV2.1.2.csv'
config['dataset_version'] = 'V2.1.2'
config['versiontag'] = '3rk9ayjc-V1'

# try! to collect same amount of samples from each location
config['balance_by_location'] = False
config['confidence_thresholds'] = {
        'biophony':0.99,
            'insect': 0.99,
            'bird':0.99,
                'songbirds':0.95,
                'duck-goose-swan': 0.9,
                'grouse-ptarmigan': 0.9,
        'anthrophony': 0.99,
            'aircraft': 0.99,
        'silence': 0.99,
    }
                             # only applies to main label



# I DID NOT USE THESE NUMBERS
config['new_data_classes_counts'] = {
    'biophony':10*3,
        'insect': 10*4,
        'bird':10**3,
            'songbirds':10*4,
            'duck-goose-swan': 10*4,
            'grouse-ptarmigan': 10*4,
    'anthrophony': 10*4,
        'aircraft': 10**4,
    'silence': 10*3,
}



import glob
from pathlib import Path

In [3]:
# # load and filter by confidence_threshold

# confident_preds_dict = load_filter_csv(
#     config['new_data_classes_counts'], config['reg_loc_per_set'],
#     config['merged_out_dir'], config['versiontag'],
#     config['confidence_threshold'], config['related_cols'],
#     config['count_from_each_year'],config['labels_threholds'])

In [4]:
preds_df = teacher.load_all_rawcsv_as_df(config['reg_loc_per_set'][0],config['merged_out_dir'],config['versiontag'])

In [5]:
# if config['balance_by_location']:
#     confident_preds_dict = teacher.pick_clips_equally_from_locations(
#         config['new_data_classes_counts'], confident_preds_dict)


In [6]:
preds_df.reset_index(inplace=True,drop=True)

In [7]:
confident_preds_dict = {}

for label_to_sample in config['new_data_classes_counts'].keys():
    print("--------",label_to_sample)
    confidence_threshold=config['confidence_thresholds'][label_to_sample]
    confident_preds = teacher.pick_confidents(preds_df, label_to_sample,
                                                     confidence_threshold,
                                                     config['related_cols'],
                                                     config['labels_thresholds'],
                                                    columns_to_exclude=['TIMESTAMP','sum','location','region'])
    confident_preds_dict[label_to_sample] = confident_preds.copy()


-------- biophony
anthrophony
1998831
aircraft
1998831
insect
1732830
silence
1699232
-------- insect
anthrophony
139804
aircraft
139804
bird
34420
songbirds
13292
duck-goose-swan
13292
grouse-ptarmigan
13292
silence
13281
-------- bird
anthrophony
1099053
aircraft
1099053
insect
1081408
silence
1056244
-------- songbirds
anthrophony
17626
aircraft
17626
duck-goose-swan
17390
grouse-ptarmigan
15261
insect
14379
silence
13957
-------- duck-goose-swan
anthrophony
51859
aircraft
51859
songbirds
9055
grouse-ptarmigan
5484
insect
4606
silence
4606
-------- grouse-ptarmigan
anthrophony
403689
aircraft
403689
songbirds
9541
duck-goose-swan
4267
insect
1953
silence
1953
-------- anthrophony
biophony
26493
bird
23769
songbirds
14934
duck-goose-swan
14934
grouse-ptarmigan
14934
insect
14934
silence
14934
-------- aircraft
biophony
35356
bird
30189
songbirds
15418
duck-goose-swan
15418
grouse-ptarmigan
15418
insect
15418
silence
15418
-------- silence
anthrophony
0
aircraft
0
biophony
0
bird
0
so

In [3]:
selected_data={}

# sort values by sum so other lables has lower confidence
for k,v in confident_preds_dict.items():
    confident_preds_dict[k]=v.sort_values(by='sum', ascending=True)
    
#select samples
for k,v in confident_preds_dict.items():
    print(k,len(v))
    if k in ['anthrophony', 'aircraft']:
        selected_data[k]=v.iloc[:4000]
    else:
        selected_data[k]=v.iloc[:2000]


NameError: name 'confident_preds_dict' is not defined

In [9]:
print('label_1','label_2','common')
label_pairs = list(itertools.combinations(confident_preds_dict.keys(), 2))
for k1,k2 in label_pairs:
    if k1!=k2:
        s1 = pd.merge(selected_data[k1], selected_data[k2], how='inner', )
        print(k1,k2,len(s1))



label_1 label_2 common
biophony insect 0
biophony bird 1964
biophony songbirds 0
biophony duck-goose-swan 0
biophony grouse-ptarmigan 0
biophony anthrophony 0
biophony aircraft 0
biophony silence 0
insect bird 0
insect songbirds 0
insect duck-goose-swan 0
insect grouse-ptarmigan 0
insect anthrophony 0
insect aircraft 0
insect silence 0
bird songbirds 0
bird duck-goose-swan 0
bird grouse-ptarmigan 0
bird anthrophony 0
bird aircraft 0
bird silence 0
songbirds duck-goose-swan 0
songbirds grouse-ptarmigan 0
songbirds anthrophony 0
songbirds aircraft 0
songbirds silence 0
duck-goose-swan grouse-ptarmigan 0
duck-goose-swan anthrophony 0
duck-goose-swan aircraft 0
duck-goose-swan silence 0
grouse-ptarmigan anthrophony 0
grouse-ptarmigan aircraft 0
grouse-ptarmigan silence 0
anthrophony aircraft 4000
anthrophony silence 0
aircraft silence 0


In [10]:
import pandas as pd
selected_data_merged=pd.concat(selected_data.values())
selected_data_merged_unq = selected_data_merged.drop_duplicates()

In [11]:
print('all samples - unique samples = diff')
all_count = len(selected_data_merged)
unq_count = len(selected_data_merged_unq)
print(all_count,unq_count,all_count-unq_count)

all samples - unique samples = diff
19953 13989 5964


In [12]:
file_properties_df = pd.read_pickle(
    '/scratch/enis/data/nna/database/allFields_dataV5.pkl')


In [13]:


new_dataset_csv, not_found_rows = teacher.generate_new_dataset(
    selected_data_merged_unq,
    config['versiontag'],
    config['split_out_path'],
    file_properties_df,
    config['upper_taxo_links'],
    config['dataset_version'],
    excell_label_headers= config['excell_label_headers'],
    labels_thresholds= config['labels_thresholds'],
    dry_run=False,
    excell_labels_2_names=config['excell_labels_2_names'],
    stereo2mono=True,
    overwrite=True,
)


# from copy import deepcopy
# new_dataset_csv_backup = deepcopy(new_dataset_csv)
# not_found_rows_backup = deepcopy(not_found_rows)

# del not_found_rows_backup,new_dataset_csv_backup


# write_csv(config['new_dataset_path'],
#           new_dataset_csv,
#           fieldnames=config['excell_all_headers'])

--------------------------------------
Not Found
3928632 TIMESTAMP           2019-07-20_04:29:52
anthrophony                       0.005
aircraft                          0.003
biophony                              1
bird                                  1
songbirds                         0.147
duck-goose-swan                   0.099
grouse-ptarmigan                   0.08
insect                            0.055
silence                           0.149
sum                               2.538
region                             anwr
location                             48
Name: 3928632, dtype: object
--------------------------------------
Not Found
7108181 TIMESTAMP           2019-05-09_19:59:52
anthrophony                       0.018
aircraft                          0.012
biophony                              1
bird                                  1
songbirds                         0.186
duck-goose-swan                   0.116
grouse-ptarmigan                  0.095
insect           

In [34]:
# new_dataset_csv

In [62]:
!/scratch/enis/conda/envs/speechEnv/bin/python /home/enis/projects/nna/src/nna/slack_message.py -t "cpu job ended" -m 'split audio ended'


In [20]:
len(new_dataset_csv)

13968

In [16]:
from copy import deepcopy
new_dataset_csv_backup = deepcopy(new_dataset_csv)
not_found_rows_backup = deepcopy(not_found_rows)




In [18]:
print(config['new_dataset_path'])
teacher.write_csv(config['new_dataset_path'],
          new_dataset_csv,
          fieldnames=config['excell_all_headers'])

./datasetV2.1.2.csv


In [19]:
!cd /home/enis/projects/nna/src/scripts

/home/enis/projects/nna/src/scripts


In [63]:
len(new_dataset_csv)

13968

In [ ]:
from nna.fileUtils import getLength,change_sampling_rate

In [61]:
for i,row in enumerate(new_dataset_csv):
    relative_part=row['Clip Path'].relative_to('/scratch/enis/data/nna/labeling/megan/3rk9ayjc-V1/')
    new_file='/scratch/enis/data/nna/labeling/megan/3rk9ayjc-V2'/ relative_part
    new_file.parent.mkdir(exist_ok=True,parents=True)
#     print(row['Clip Path'],'\n',new_file)
#     break
    output=change_sampling_rate(row['Clip Path'],new_file,48000)
#     break
    if i%1000==0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000


In [68]:
short_ones=[]
for i,row in enumerate(new_dataset_csv):
    length = getLength(row['Clip Path'])
    if length!=10.0:
        short_ones.append(row)
    if i%1000==0:
        print(i)

0
1000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000


In [71]:
for row in short_ones:
    print(row['Clip Path'])
    length = getLength(row['Clip Path'])
    print(length)

/scratch/enis/data/nna/labeling/megan/3rk9ayjc-V1/dalton/05/S4A10454_20190915_144000_0m_20s__0m_30s.wav
9.999833
/scratch/enis/data/nna/labeling/megan/3rk9ayjc-V1/dalton/05/S4A10454_20190829_172000_1m_10s__1m_20s.wav
10.000167
/scratch/enis/data/nna/labeling/megan/3rk9ayjc-V1/dalton/05/S4A10454_20190902_224000_0m_10s__0m_20s.wav
9.999833
/scratch/enis/data/nna/labeling/megan/3rk9ayjc-V1/dalton/05/S4A10454_20190427_031814_1m_0s__1m_10s.wav
10.000167
/scratch/enis/data/nna/labeling/megan/3rk9ayjc-V1/dalton/03/S4A10437_20190423_130755_0m_0s__0m_10s.wav
9.999833
